# Delta Operations & Medallion Architecture - Workshop

**Cel szkoleniowy:** Praktyczne zastosowanie operacji Delta Lake (CRUD, MERGE, Time Travel, OPTIMIZE) oraz implementacja architektury Bronze/Silver/Gold.

**Zakres tematyczny:**
- Delta Lake CRUD operations (CREATE, INSERT, UPDATE, DELETE, MERGE)
- Time Travel i wersjonowanie
- Projektowanie warstw Medallion (Bronze/Silver/Gold)
- Optymalizacja tabel (OPTIMIZE, ZORDER, VACUUM)
- Audit metadata i data quality checks

**Czas trwania:** 90 minut

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 2 - Lakehouse & Delta Lake
- **Typ notebooka**: Workshop
- **Wymagania techniczne**:
  - Databricks Runtime 13.0+ (zalecane: 14.3 LTS)
  - Unity Catalog włączony
  - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Klaster: Standard z minimum 2 workers

## Izolacja per użytkownik

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [ ]:
%run ../../00_setup

## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime, timedelta

# Wyświetl kontekst użytkownika
print("=== Kontekst użytkownika ===")
print(f"Katalog: {CATALOG}")
print(f"Schema Bronze: {BRONZE_SCHEMA}")
print(f"Schema Silver: {SILVER_SCHEMA}")
print(f"Schema Gold: {GOLD_SCHEMA}")
print(f"Użytkownik: {raw_user}")

# Ustaw katalog jako domyślny
spark.sql(f"USE CATALOG {CATALOG}")

# Ścieżki do danych źródłowych - TODO: Uzupełnij brakujące ścieżki
ORDERS_JSON = f"{DATASET_BASE_PATH}/____/orders_batch.json"  # TODO: Uzupełnij folder
CUSTOMERS_CSV = f"{DATASET_BASE_PATH}/customers/____.csv"    # TODO: Uzupełnij nazwę pliku
PRODUCTS_PARQUET = f"{____}/products/products.parquet"        # TODO: Uzupełnij bazową ścieżkę

print(f"\n=== Ścieżki do danych ===")
print(f"Orders: {ORDERS_JSON}")
print(f"Customers: {CUSTOMERS_CSV}")
print(f"Products: {PRODUCTS_PARQUET}")

---

## Zadanie 1: Delta Lake CRUD Operations (20 minut)

**Cel:** Praktyczne zastosowanie podstawowych operacji Delta Lake: CREATE, INSERT, UPDATE, DELETE.

### Zadanie 1.1: Utworzenie tabeli Delta z danymi produktów

**Instrukcje:**
1. Wczytaj dane z pliku `products.parquet` z folderu `dataset/products/`
2. Dodaj kolumnę `load_timestamp` z aktualnym czasem
3. Zapisz jako tabelę Delta w schemacie Bronze: `products_bronze`
4. Użyj mode `overwrite` i włącz `overwriteSchema`

**Oczekiwany rezultat:**
- Tabela `products_bronze` utworzona w schemacie Bronze
- Wszystkie rekordy załadowane z audit timestamp

In [ ]:
# TODO: Zadanie 1.1 - Utworzenie tabeli Delta

spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

# Wczytaj dane produktów - TODO: Uzupełnij format i ścieżkę
products_df = (
    spark.read
    .format("____")  # TODO: Uzupełnij format (parquet)
    .load(____)      # TODO: Uzupełnij ścieżkę (PRODUCTS_PARQUET)
)

print("=== Dane produktów ===")
products_df.printSchema()
display(products_df.limit(5))

# Dodaj audit column: load_timestamp - TODO: Uzupełnij funkcję
products_with_audit = products_df.withColumn("load_timestamp", F.____())  # TODO: current_timestamp

# Zapisz jako tabelę Delta - TODO: Uzupełnij brakujące parametry
products_table = f"{BRONZE_SCHEMA}.products_bronze"

(
    products_with_audit
    .write
    .format("____")              # TODO: Uzupełnij format (delta)
    .mode("____")                # TODO: Uzupełnij mode (overwrite)
    .option("____", "true")      # TODO: Uzupełnij opcję (overwriteSchema)
    .saveAsTable(____)           # TODO: Uzupełnij nazwę tabeli (products_table)
)

print(f"\n✓ Utworzono tabelę: {products_table}")
print(f"Liczba rekordów: {spark.table(products_table).count()}")

### Zadanie 1.2: UPDATE - Aktualizacja cen produktów

**Instrukcje:**
1. Zaktualizuj cenę (`unit_price`) wszystkich produktów z kategorii "Electronics" - zwiększ o 10%
2. Użyj SQL UPDATE statement
3. Wyświetl zaktualizowane rekordy

**Wskazówki:**
- Użyj: `UPDATE table_name SET column = value WHERE condition`
- Zwiększenie o 10%: `unit_price * 1.1`

In [ ]:
# TODO: Zadanie 1.2 - UPDATE cen produktów

# Sprawdź dane przed UPDATE
print("=== Produkty Electronics PRZED aktualizacją ===")
display(
    spark.table(products_table)
    .filter(F.col("category") == "Electronics")
    .select("product_id", "product_name", "category", "unit_price")
)

# UPDATE: zwiększ cenę o 10% dla kategorii Electronics
# TODO: Uzupełnij brakujące części SQL
spark.sql(f"""
    ____ {products_table}           -- TODO: Uzupełnij komendę (UPDATE)
    SET unit_price = ____ * ____    -- TODO: Uzupełnij wyrażenie (unit_price * 1.1)
    WHERE ____ = '____'             -- TODO: Uzupełnij warunek (category = 'Electronics')
""")

# Sprawdź dane po UPDATE
print("\n=== Produkty Electronics PO aktualizacji ===")
display(
    spark.table(products_table)
    .filter(F.col("category") == "Electronics")
    .select("product_id", "product_name", "category", "unit_price")
)

print("✓ Ceny zaktualizowane!")

### Zadanie 1.3: DELETE - Usunięcie produktów

**Instrukcje:**
1. Usuń produkty, które mają `stock_quantity = 0` (brak w magazynie)
2. Użyj SQL DELETE statement
3. Wyświetl liczbę usuniętych rekordów

**Wskazówki:**
- Użyj: `DELETE FROM table_name WHERE condition`

In [ ]:
# TODO: Zadanie 1.3 - DELETE produktów bez stanu magazynowego

# Liczba rekordów przed DELETE
count_before = spark.table(products_table).count()
out_of_stock_count = spark.table(products_table).filter(F.col("stock_quantity") == 0).count()

print(f"=== Stan przed DELETE ===")
print(f"Liczba produktów: {count_before}")
print(f"Produkty bez stanu (stock_quantity = 0): {out_of_stock_count}")

# DELETE produktów bez stanu magazynowego
# TODO: Uzupełnij SQL DELETE statement
spark.sql(f"""
    ____ FROM {products_table}  -- TODO: Uzupełnij komendę (DELETE)
    WHERE ____ = ____           -- TODO: Uzupełnij warunek (stock_quantity = 0)
""")

# Liczba rekordów po DELETE
count_after = spark.table(products_table).count()

print(f"\n=== Stan po DELETE ===")
print(f"Liczba produktów: {count_after}")
print(f"Usunięto: {count_before - count_after} produktów")

print("✓ Produkty bez stanu usunięte!")

---

## Zadanie 2: MERGE INTO - Upsert Operations (15 minut)

**Cel:** Implementacja operacji MERGE dla upsert (update existing + insert new).

### Zadanie 2.1: MERGE nowych zamówień

**Instrukcje:**
1. Wczytaj dane zamówień z `orders_batch.json`
2. Utwórz tabelę `orders_bronze` jeśli nie istnieje
3. Użyj MERGE INTO do załadowania danych:
   - MATCHED: zaktualizuj `order_status` i `order_amount`
   - NOT MATCHED: wstaw nowy rekord
4. Klucz: `order_id`

**Oczekiwany rezultat:**
- Istniejące zamówienia zaktualizowane
- Nowe zamówienia dodane

In [ ]:
# TODO: Zadanie 2.1 - MERGE INTO dla zamówień

spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

# Wczytaj nowe dane zamówień - TODO: Uzupełnij format i opcje
new_orders = (
    spark.read
    .format("____")               # TODO: Uzupełnij format (json)
    .option("____", "true")       # TODO: Uzupełnij opcję (multiLine)
    .load(____)                   # TODO: Uzupełnij ścieżkę (ORDERS_JSON)
    .withColumn("ingest_timestamp", F.current_timestamp())
    .withColumn("ingested_by", F.lit(raw_user))
)

print("=== Nowe zamówienia do merge ===")
display(new_orders.limit(5))

# Nazwa tabeli docelowej
orders_table = f"{BRONZE_SCHEMA}.orders_bronze"

# Utwórz tabelę jeśli nie istnieje (initial load)
if not spark.catalog.tableExists(orders_table):
    new_orders.limit(0).write.format("delta").saveAsTable(orders_table)
    print(f"✓ Utworzono pustą tabelę: {orders_table}")

# Zarejestruj DataFrame jako temp view dla MERGE
new_orders.createOrReplaceTempView("new_orders_view")

# Liczba przed MERGE
count_before = spark.table(orders_table).count()

# MERGE INTO - TODO: Uzupełnij brakujące części
spark.sql(f"""
    MERGE INTO {orders_table} AS target
    USING new_orders_view AS source
    ON target.____ = source.____                    -- TODO: Uzupełnij klucz (order_id)
    WHEN ____ THEN                                  -- TODO: Uzupełnij (MATCHED)
        UPDATE SET
            target.order_status = source.____,      -- TODO: Uzupełnij kolumnę
            target.order_amount = source.____,      -- TODO: Uzupełnij kolumnę
            target.ingest_timestamp = source.ingest_timestamp
    WHEN ____ ____ THEN                             -- TODO: Uzupełnij (NOT MATCHED)
        INSERT *
""")

# Liczba po MERGE
count_after = spark.table(orders_table).count()

print(f"\n=== Wynik MERGE ===")
print(f"Rekordy przed: {count_before}")
print(f"Rekordy po: {count_after}")
print(f"Dodano nowych: {count_after - count_before}")

print("✓ MERGE zakończony!")

---

## Zadanie 3: Time Travel (10 minut)

**Cel:** Wykorzystanie Time Travel do przeglądania historii zmian i przywracania danych.

### Zadanie 3.1: Historia wersji tabeli

**Instrukcje:**
1. Wyświetl historię tabeli `products_bronze` używając `DESCRIBE HISTORY`
2. Znajdź wersję przed wykonaniem UPDATE (Zadanie 1.2)
3. Odczytaj dane z tej wersji używając `VERSION AS OF`

**Wskazówki:**
- `DESCRIBE HISTORY table_name`
- `SELECT * FROM table_name VERSION AS OF version_number`

In [ ]:
# TODO: Zadanie 3.1 - Time Travel

# Wyświetl historię wersji tabeli - TODO: Uzupełnij komendę
print("=== Historia wersji tabeli products_bronze ===")
history = spark.sql(f"____ ____ {products_table}")  # TODO: DESCRIBE HISTORY
display(history.select("version", "timestamp", "operation", "operationMetrics"))

# Znajdź wersję przed UPDATE (operation = 'WRITE' na początku)
first_version = history.filter(F.col("operation") == "WRITE").orderBy("version").first()["version"]

print(f"\n=== Odczytanie danych z wersji {first_version} (przed UPDATE) ===")

# Time Travel: odczytaj dane z konkretnej wersji
# TODO: Uzupełnij brakujące części SQL
products_old_version = spark.sql(f"""
    SELECT product_id, product_name, category, unit_price
    FROM {products_table} 
    ____ AS OF ____                    -- TODO: Uzupełnij (VERSION AS OF {first_version})
    WHERE category = 'Electronics'
""")

display(products_old_version)

# Porównaj z aktualną wersją
print("\n=== Aktualna wersja (po UPDATE) ===")
products_current = spark.sql(f"""
    SELECT product_id, product_name, category, unit_price
    FROM {products_table}
    WHERE category = 'Electronics'
""")
display(products_current)

print("✓ Time Travel działa!")

---

## Zadanie 4: Implementacja Medallion Architecture (30 minut)

**Cel:** Zbudowanie pipeline'u Bronze → Silver → Gold zgodnie z architekturą medalionową.

### Zadanie 4.1: Bronze Layer - Raw Data Landing

**Instrukcje:**
1. Wczytaj dane klientów z `customers.csv`
2. Dodaj audit metadata:
   - `ingest_timestamp` (current_timestamp)
   - `source_file` (input_file_name)
   - `ingested_by` (raw_user)
3. Zapisz jako `customers_bronze` w Bronze schema

**Oczekiwany rezultat:**
- Tabela Bronze z surowymi danymi + audit columns

In [ ]:
# TODO: Zadanie 4.1 - Bronze Layer

spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

# Wczytaj dane klientów - TODO: Uzupełnij format i opcje
customers_raw = (
    spark.read
    .format("____")              # TODO: Uzupełnij format (csv)
    .option("____", "true")      # TODO: Uzupełnij opcję (header)
    .option("____", "true")      # TODO: Uzupełnij opcję (inferSchema)
    .load(____)                  # TODO: Uzupełnij ścieżkę (CUSTOMERS_CSV)
)

print("=== Surowe dane klientów ===")
customers_raw.printSchema()
display(customers_raw.limit(5))

# Dodaj audit metadata dla Bronze - TODO: Uzupełnij kolumny i funkcje
customers_bronze = (
    customers_raw
    .withColumn("____", F.____())           # TODO: ingest_timestamp, current_timestamp()
    .withColumn("____", F.____())           # TODO: source_file, input_file_name()
    .withColumn("____", F.lit(____))        # TODO: ingested_by, raw_user
)

# Zapisz do Bronze
customers_bronze_table = f"{BRONZE_SCHEMA}.customers_bronze"

(
    customers_bronze
    .write
    .format("____")              # TODO: delta
    .mode("____")                # TODO: overwrite
    .option("overwriteSchema", "true")
    .saveAsTable(____)           # TODO: customers_bronze_table
)

print(f"\n✓ Bronze layer utworzony: {customers_bronze_table}")
print(f"Liczba rekordów: {spark.table(customers_bronze_table).count()}")

### Zadanie 4.2: Silver Layer - Cleansed & Validated

**Instrukcje:**
1. Wczytaj dane z Bronze: `customers_bronze`
2. Zastosuj transformacje:
   - Usuń duplikaty po `customer_id`
   - Odfiltruj rekordy gdzie `customer_id` lub `email` jest NULL
   - Standaryzuj `email` do lowercase
   - Waliduj: `age` musi być > 0 i < 120
   - Dodaj kolumnę `silver_processed_timestamp`
   - Dodaj kolumnę `data_quality_flag` = "VALID"
3. Zapisz jako `customers_silver` w Silver schema

**Oczekiwany rezultat:**
- Oczyszczone, zwalidowane dane w Silver

In [ ]:
# TODO: Zadanie 4.2 - Silver Layer

spark.sql(f"USE SCHEMA {SILVER_SCHEMA}")

# Wczytaj dane z Bronze
customers_bronze_df = spark.table(customers_bronze_table)

# Silver transformations - TODO: Uzupełnij brakujące transformacje
customers_silver = (
    customers_bronze_df
    # Deduplikacja - TODO: Uzupełnij kolumnę
    .dropDuplicates([____])                    # TODO: customer_id
    
    # Walidacja NOT NULL - TODO: Uzupełnij warunki
    .filter(F.col("____").isNotNull())         # TODO: customer_id
    .filter(F.col("____").isNotNull())         # TODO: email
    
    # Standaryzacja email - TODO: Uzupełnij funkcję
    .withColumn("email", F.____(F.col("email")))  # TODO: lower
    
    # Walidacja wieku - TODO: Uzupełnij warunki
    .filter((F.col("____") > ____) & (F.col("____") < ____))  # TODO: age > 0, age < 120
    
    # Silver metadata - TODO: Uzupełnij kolumny
    .withColumn("____", F.current_timestamp())     # TODO: silver_processed_timestamp
    .withColumn("____", F.lit("____"))             # TODO: data_quality_flag, "VALID"
)

print("=== Silver layer - cleansed data ===")
display(customers_silver.limit(5))

# Zapisz do Silver
customers_silver_table = f"{SILVER_SCHEMA}.customers_silver"

(
    customers_silver
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(customers_silver_table)
)

bronze_count = customers_bronze_df.count()
silver_count = spark.table(customers_silver_table).count()

print(f"\n✓ Silver layer utworzony: {customers_silver_table}")
print(f"Bronze records: {bronze_count}")
print(f"Silver records: {silver_count}")
print(f"Filtered out: {bronze_count - silver_count} records")

### Zadanie 4.3: Gold Layer - Business Aggregates

**Instrukcje:**
1. Wczytaj dane z Silver: `customers_silver`
2. Utwórz agregację klientów per kraj:
   - `customer_count`: liczba klientów
   - `avg_age`: średni wiek
   - `min_registration_date`: najwcześniejsza rejestracja
   - `max_registration_date`: najpóźniejsza rejestracja
3. Dodaj `gold_created_timestamp`
4. Zapisz jako `customer_summary_by_country` w Gold schema

**Oczekiwany rezultat:**
- Tabela Gold z agregacjami biznesowymi

In [ ]:
# TODO: Zadanie 4.3 - Gold Layer

spark.sql(f"USE SCHEMA {GOLD_SCHEMA}")

# Wczytaj dane z Silver
customers_silver_df = spark.table(customers_silver_table)

# Gold aggregation: Summary per country
# TODO: Uzupełnij brakujące części agregacji
customer_summary = (
    customers_silver_df
    .groupBy("____")                # TODO: country
    .agg(
        F.count("____").alias("____"),      # TODO: customer_id, customer_count
        F.avg("____").alias("____"),        # TODO: age, avg_age
        F.min("____").alias("____"),        # TODO: registration_date, min_registration_date
        F.max("____").alias("____")         # TODO: registration_date, max_registration_date
    )
    .withColumn("____", F.current_timestamp())  # TODO: gold_created_timestamp
    .orderBy("customer_count", ascending=False)
)

print("=== Gold layer - Customer Summary by Country ===")
display(customer_summary)

# Zapisz do Gold
gold_table = f"{GOLD_SCHEMA}.customer_summary_by_country"

(
    customer_summary
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(gold_table)
)

print(f"\n✓ Gold layer utworzony: {gold_table}")
print(f"Liczba krajów: {spark.table(gold_table).count()}")

---

## Zadanie 5: Optymalizacja tabel Delta (15 minut)

**Cel:** Zastosowanie technik optymalizacji: OPTIMIZE, ZORDER, VACUUM.

### Zadanie 5.1: OPTIMIZE - Compaction małych plików

**Instrukcje:**
1. Sprawdź liczbę plików w tabeli `customers_silver` używając `DESCRIBE DETAIL`
2. Uruchom `OPTIMIZE` na tabeli
3. Sprawdź ponownie liczbę plików - powinna się zmniejszyć

**Wskazówki:**
- `DESCRIBE DETAIL table_name`
- `OPTIMIZE table_name`

In [ ]:
# TODO: Zadanie 5.1 - OPTIMIZE

# Stan przed OPTIMIZE - TODO: Uzupełnij komendę
print("=== Stan przed OPTIMIZE ===")
detail_before = spark.sql(f"____ ____ {customers_silver_table}")  # TODO: DESCRIBE DETAIL
display(detail_before.select("numFiles", "sizeInBytes"))

num_files_before = detail_before.collect()[0]["numFiles"]
print(f"Liczba plików przed: {num_files_before}")

# Uruchom OPTIMIZE - TODO: Uzupełnij komendę
print("\n=== Uruchamianie OPTIMIZE ===")
optimize_result = spark.sql(f"____ {____}")  # TODO: OPTIMIZE {customers_silver_table}
display(optimize_result)

# Stan po OPTIMIZE
print("\n=== Stan po OPTIMIZE ===")
detail_after = spark.sql(f"DESCRIBE DETAIL {customers_silver_table}")
display(detail_after.select("numFiles", "sizeInBytes"))

num_files_after = detail_after.collect()[0]["numFiles"]
print(f"Liczba plików po: {num_files_after}")
print(f"Redukcja: {num_files_before - num_files_after} plików")

print("✓ OPTIMIZE zakończony!")

### Zadanie 5.2: ZORDER - Clustering dla lepszego data skipping

**Instrukcje:**
1. Uruchom `OPTIMIZE` z `ZORDER BY` na tabeli `orders_bronze`
2. Użyj `ZORDER BY (customer_id)` - kolumna często używana w filtrach
3. Sprawdź metryki optymalizacji

**Wskazówki:**
- `OPTIMIZE table_name ZORDER BY (column1, column2)`

In [ ]:
# TODO: Zadanie 5.2 - ZORDER

# Uruchom OPTIMIZE z ZORDER na customer_id
# TODO: Uzupełnij komendę SQL
print("=== OPTIMIZE + ZORDER BY customer_id ===")

zorder_result = spark.sql(f"""
    ____ {orders_table}         -- TODO: OPTIMIZE
    ____ BY (____)              -- TODO: ZORDER BY (customer_id)
""")

display(zorder_result)

print("✓ ZORDER zakończony - dane są teraz lepiej zorganizowane dla queries po customer_id!")

---

## Walidacja i weryfikacja

### Checklist - Co powinieneś uzyskać:
- [ ] Tabela `products_bronze` utworzona z operacjami UPDATE i DELETE
- [ ] Tabela `orders_bronze` z zastosowanym MERGE INTO
- [ ] Time Travel działa - można odczytać starą wersję `products_bronze`
- [ ] Pipeline Medallion: `customers_bronze` → `customers_silver` → `customer_summary_by_country`
- [ ] Tabele zoptymalizowane (OPTIMIZE, ZORDER)

### Komendy weryfikacyjne:

In [ ]:
# Weryfikacja wyników

print("=== WERYFIKACJA WYNIKÓW ===\n")

# 1. Sprawdź tabele Bronze
print("1. Tabele Bronze:")
bronze_tables = spark.sql(f"SHOW TABLES IN {BRONZE_SCHEMA}").filter(F.col("tableName").like("%bronze%"))
display(bronze_tables)

# 2. Sprawdź tabele Silver
print("\n2. Tabele Silver:")
silver_tables = spark.sql(f"SHOW TABLES IN {SILVER_SCHEMA}").filter(F.col("tableName").like("%silver%"))
display(silver_tables)

# 3. Sprawdź tabele Gold
print("\n3. Tabele Gold:")
gold_tables = spark.sql(f"SHOW TABLES IN {GOLD_SCHEMA}")
display(gold_tables)

# 4. Sprawdź historię dla products_bronze
print("\n4. Historia products_bronze (powinny być operacje: WRITE, UPDATE, DELETE):")
history = spark.sql(f"DESCRIBE HISTORY {products_table}")
display(history.select("version", "timestamp", "operation", "operationMetrics").limit(10))

# 5. Sprawdź dane Gold
print("\n5. Przykładowe dane Gold:")
display(spark.table(gold_table).limit(10))

print("\n✓ Wszystkie testy przeszły pomyślnie!")

---

## Podsumowanie

**W tym warsztacie nauczyłeś się:**

✅ **Delta Lake CRUD Operations:**
- CREATE TABLE z różnych formatów źródłowych
- UPDATE - aktualizacja rekordów
- DELETE - usuwanie rekordów
- MERGE INTO - upsert (update + insert)

✅ **Time Travel:**
- DESCRIBE HISTORY - historia wersji
- VERSION AS OF - odczytywanie starych wersji
- Możliwość audytu i rollback

✅ **Medallion Architecture:**
- Bronze: Raw data + audit metadata (immutable, append-only)
- Silver: Cleansed, validated, deduplicated
- Gold: Business aggregates, KPI models

✅ **Optymalizacja:**
- OPTIMIZE - compaction małych plików
- ZORDER - clustering dla lepszego data skipping
- DESCRIBE DETAIL - monitoring stanu tabel

**Kluczowe wnioski:**
1. Delta Lake zapewnia ACID transactions dla data lake
2. Time Travel umożliwia audyt i recovery bez backupów
3. Medallion Architecture separuje raw data od business logic
4. Regularna optymalizacja (OPTIMIZE) jest kluczowa dla performance

**Następne kroki:**
- **Kolejny warsztat**: 02_ingestion_pipeline_workshop.ipynb - COPY INTO, Auto Loader, Streaming
- **Materiały dodatkowe**: Delta Lake Documentation, Medallion Architecture Guide

---

## Cleanup

Opcjonalnie: usuń utworzone tabele po zakończeniu warsztatu:

In [ ]:
# Opcjonalne czyszczenie zasobów testowych
# UWAGA: Uruchom tylko jeśli chcesz usunąć wszystkie utworzone dane

# spark.sql(f"DROP TABLE IF EXISTS {products_table}")
# spark.sql(f"DROP TABLE IF EXISTS {orders_table}")
# spark.sql(f"DROP TABLE IF EXISTS {customers_bronze_table}")
# spark.sql(f"DROP TABLE IF EXISTS {customers_silver_table}")
# spark.sql(f"DROP TABLE IF EXISTS {gold_table}")

# spark.catalog.clearCache()
# print("Zasoby zostały wyczyszczone")

In [ ]:
```xml
<VSCode.Cell language="markdown">
# Delta Operations & Medallion Architecture - Workshop

**Cel szkoleniowy:** Praktyczne zastosowanie operacji Delta Lake (CRUD, MERGE, Time Travel, OPTIMIZE) oraz implementacja architektury Bronze/Silver/Gold.

**Zakres tematyczny:**
- Delta Lake CRUD operations (CREATE, INSERT, UPDATE, DELETE, MERGE)
- Time Travel i wersjonowanie
- Projektowanie warstw Medallion (Bronze/Silver/Gold)
- Optymalizacja tabel (OPTIMIZE, ZORDER, VACUUM)
- Audit metadata i data quality checks

**Czas trwania:** 90 minut
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 2 - Lakehouse & Delta Lake
- **Typ notebooka**: Workshop
- **Wymagania techniczne**:
  - Databricks Runtime 13.0+ (zalecane: 14.3 LTS)
  - Unity Catalog włączony
  - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Klaster: Standard z minimum 2 workers
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Izolacja per użytkownik

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:
</VSCode.Cell>
<VSCode.Cell language="python">
%run ../../00_setup
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:
</VSCode.Cell>
<VSCode.Cell language="python">
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime, timedelta

# Wyświetl kontekst użytkownika
print("=== Kontekst użytkownika ===")
print(f"Katalog: {CATALOG}")
print(f"Schema Bronze: {BRONZE_SCHEMA}")
print(f"Schema Silver: {SILVER_SCHEMA}")
print(f"Schema Gold: {GOLD_SCHEMA}")
print(f"Użytkownik: {raw_user}")

# Ustaw katalog jako domyślny
spark.sql(f"USE CATALOG {CATALOG}")

# Ścieżki do danych źródłowych
ORDERS_JSON = f"{DATASET_BASE_PATH}/orders/orders_batch.json"
CUSTOMERS_CSV = f"{DATASET_BASE_PATH}/customers/customers.csv"

print(f"\n=== Ścieżki do danych ===")
print(f"Orders: {ORDERS_JSON}")
print(f"Customers: {CUSTOMERS_CSV}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Zadanie 1: Delta Lake CRUD Operations (20 minut)

**Cel:** Praktyczne zastosowanie podstawowych operacji Delta Lake: CREATE, INSERT, UPDATE, DELETE.

### Zadanie 1.1: Utworzenie tabeli Delta z danymi produktów

**Instrukcje:**
1. Wczytaj dane z pliku `products.parquet` z folderu `dataset/products/`
2. Dodaj kolumnę `load_timestamp` z aktualnym czasem
3. Zapisz jako tabelę Delta w schemacie Bronze: `products_bronze`
4. Użyj mode `overwrite` i włącz `overwriteSchema`

**Oczekiwany rezultat:**
- Tabela `products_bronze` utworzona w schemacie Bronze
- Wszystkie rekordy załadowane z audit timestamp
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 1.1 - Utworzenie tabeli Delta

spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

# Ścieżka do pliku produktów
PRODUCTS_PARQUET = f"{____}/products/products.parquet"

# Wczytaj dane produktów
products_df = (
    spark.read
    .format("____")
    .load(____)
)

print("=== Dane produktów ===")
products_df.printSchema()
display(products_df.limit(5))

# Dodaj audit column: load_timestamp
products_with_audit = products_df.withColumn("____", F.____)

# Zapisz jako tabelę Delta
products_table = f"{BRONZE_SCHEMA}.products_bronze"

(
    products_with_audit
    .write
    .format("____")
    .mode("____")
    .option("overwriteSchema", "____")
    .saveAsTable(____)
)

print(f"\n✓ Utworzono tabelę: {products_table}")
print(f"Liczba rekordów: {spark.table(products_table).count()}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
### Zadanie 1.2: UPDATE - Aktualizacja cen produktów

**Instrukcje:**
1. Zaktualizuj cenę (`unit_price`) wszystkich produktów z kategorii "Electronics" - zwiększ o 10%
2. Użyj SQL UPDATE statement
3. Wyświetl zaktualizowane rekordy

**Wskazówki:**
- Użyj: `UPDATE table_name SET column = value WHERE condition`
- Zwiększenie o 10%: `unit_price * 1.1`
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 1.2 - UPDATE cen produktów

# Sprawdź dane przed UPDATE
print("=== Produkty Electronics PRZED aktualizacją ===")
display(
    spark.table(products_table)
    .filter(F.col("category") == "Electronics")
    .select("product_id", "product_name", "category", "unit_price")
)

# UPDATE: zwiększ cenę o 10% dla kategorii Electronics
spark.sql(f"""
    ____ {products_table}
    SET ____ = unit_price * ____
    WHERE ____ = '____'
""")

# Sprawdź dane po UPDATE
print("\n=== Produkty Electronics PO aktualizacji ===")
display(
    spark.table(products_table)
    .filter(F.col("category") == "Electronics")
    .select("product_id", "product_name", "category", "unit_price")
)

print("✓ Ceny zaktualizowane!")
</VSCode.Cell>
<VSCode.Cell language="markdown">
### Zadanie 1.3: DELETE - Usunięcie produktów

**Instrukcje:**
1. Usuń produkty, które mają `stock_quantity = 0` (brak w magazynie)
2. Użyj SQL DELETE statement
3. Wyświetl liczbę usuniętych rekordów

**Wskazówki:**
- Użyj: `DELETE FROM table_name WHERE condition`
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 1.3 - DELETE produktów bez stanu magazynowego

# Liczba rekordów przed DELETE
count_before = spark.table(products_table).count()
out_of_stock_count = spark.table(products_table).filter(F.col("stock_quantity") == 0).count()

print(f"=== Stan przed DELETE ===")
print(f"Liczba produktów: {count_before}")
print(f"Produkty bez stanu (stock_quantity = 0): {out_of_stock_count}")

# DELETE produktów bez stanu magazynowego
spark.sql(f"""
    ____ FROM {products_table}
    WHERE ____ = ____
""")

# Liczba rekordów po DELETE
count_after = spark.table(products_table).count()

print(f"\n=== Stan po DELETE ===")
print(f"Liczba produktów: {count_after}")
print(f"Usunięto: {count_before - count_after} produktów")

print("✓ Produkty bez stanu usunięte!")
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Zadanie 2: MERGE INTO - Upsert Operations (15 minut)

**Cel:** Implementacja operacji MERGE dla upsert (update existing + insert new).

### Zadanie 2.1: MERGE nowych zamówień

**Instrukcje:**
1. Wczytaj dane zamówień z `orders_batch.json`
2. Utwórz tabelę `orders_bronze` jeśli nie istnieje
3. Użyj MERGE INTO do załadowania danych:
   - MATCHED: zaktualizuj `order_status` i `order_amount`
   - NOT MATCHED: wstaw nowy rekord
4. Klucz: `order_id`

**Oczekiwany rezultat:**
- Istniejące zamówienia zaktualizowane
- Nowe zamówienia dodane
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 2.1 - MERGE INTO dla zamówień

spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

# Wczytaj nowe dane zamówień
new_orders = (
    spark.read
    .format("____")
    .option("multiLine", "____")
    .load(____)
    .withColumn("ingest_timestamp", F.current_timestamp())
    .withColumn("ingested_by", F.lit(raw_user))
)

print("=== Nowe zamówienia do merge ===")
display(new_orders.limit(5))

# Nazwa tabeli docelowej
orders_table = f"{BRONZE_SCHEMA}.orders_bronze"

# Utwórz tabelę jeśli nie istnieje (initial load)
if not spark.catalog.tableExists(orders_table):
    new_orders.limit(0).write.format("delta").saveAsTable(orders_table)
    print(f"✓ Utworzono pustą tabelę: {orders_table}")

# Zarejestruj DataFrame jako temp view dla MERGE
new_orders.createOrReplaceTempView("new_orders_view")

# Liczba przed MERGE
count_before = spark.table(orders_table).count()

# MERGE INTO
spark.sql(f"""
    MERGE INTO {orders_table} AS target
    USING new_orders_view AS source
    ON target.____ = source.____
    WHEN ____ THEN
        UPDATE SET
            target.order_status = source.____,
            target.order_amount = source.____,
            target.ingest_timestamp = source.____
    WHEN ____ ____ THEN
        INSERT *
""")

# Liczba po MERGE
count_after = spark.table(orders_table).count()

print(f"\n=== Wynik MERGE ===")
print(f"Rekordy przed: {count_before}")
print(f"Rekordy po: {count_after}")
print(f"Dodano nowych: {count_after - count_before}")

print("✓ MERGE zakończony!")
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Zadanie 3: Time Travel (10 minut)

**Cel:** Wykorzystanie Time Travel do przeglądania historii zmian i przywracania danych.

### Zadanie 3.1: Historia wersji tabeli

**Instrukcje:**
1. Wyświetl historię tabeli `products_bronze` używając `DESCRIBE HISTORY`
2. Znajdź wersję przed wykonaniem UPDATE (Zadanie 1.2)
3. Odczytaj dane z tej wersji używając `VERSION AS OF`

**Wskazówki:**
- `DESCRIBE HISTORY table_name`
- `SELECT * FROM table_name VERSION AS OF version_number`
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 3.1 - Time Travel

# Wyświetl historię wersji tabeli
print("=== Historia wersji tabeli products_bronze ===")
history = spark.sql(f"____ ____ {products_table}")
display(history.select("version", "timestamp", "operation", "operationMetrics"))

# Znajdź wersję przed UPDATE (operation = 'WRITE' na początku)
first_version = history.filter(F.col("operation") == "WRITE").orderBy("version").first()["version"]

print(f"\n=== Odczytanie danych z wersji {first_version} (przed UPDATE) ===")

# Time Travel: odczytaj dane z konkretnej wersji
products_old_version = spark.sql(f"""
    SELECT product_id, product_name, category, unit_price
    FROM {products_table} 
    ____ AS OF ____
    WHERE category = 'Electronics'
""")

display(products_old_version)

# Porównaj z aktualną wersją
print("\n=== Aktualna wersja (po UPDATE) ===")
products_current = spark.sql(f"""
    SELECT product_id, product_name, category, unit_price
    FROM {products_table}
    WHERE category = 'Electronics'
""")
display(products_current)

print("✓ Time Travel działa!")
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Zadanie 4: Implementacja Medallion Architecture (30 minut)

**Cel:** Zbudowanie pipeline'u Bronze → Silver → Gold zgodnie z architekturą medalionową.

### Zadanie 4.1: Bronze Layer - Raw Data Landing

**Instrukcje:**
1. Wczytaj dane klientów z `customers.csv`
2. Dodaj audit metadata:
   - `ingest_timestamp` (current_timestamp)
   - `source_file` (input_file_name)
   - `ingested_by` (raw_user)
3. Zapisz jako `customers_bronze` w Bronze schema

**Oczekiwany rezultat:**
- Tabela Bronze z surowymi danymi + audit columns
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 4.1 - Bronze Layer

spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

# Wczytaj dane klientów
customers_raw = (
    spark.read
    .format("____")
    .option("header", "____")
    .option("inferSchema", "____")
    .load(____)
)

print("=== Surowe dane klientów ===")
customers_raw.printSchema()
display(customers_raw.limit(5))

# Dodaj audit metadata dla Bronze
customers_bronze = (
    customers_raw
    .withColumn("____", F.current_timestamp())
    .withColumn("____", F.input_file_name())
    .withColumn("____", F.lit(____))
)

# Zapisz do Bronze
customers_bronze_table = f"{BRONZE_SCHEMA}.customers_bronze"

(
    customers_bronze
    .write
    .format("____")
    .mode("____")
    .option("overwriteSchema", "true")
    .saveAsTable(____)
)

print(f"\n✓ Bronze layer utworzony: {customers_bronze_table}")
print(f"Liczba rekordów: {spark.table(customers_bronze_table).count()}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
### Zadanie 4.2: Silver Layer - Cleansed & Validated

**Instrukcje:**
1. Wczytaj dane z Bronze: `customers_bronze`
2. Zastosuj transformacje:
   - Usuń duplikaty po `customer_id`
   - Odfiltruj rekordy gdzie `customer_id` lub `email` jest NULL
   - Standaryzuj `email` do lowercase
   - Waliduj: `age` musi być > 0 i < 120
   - Dodaj kolumnę `silver_processed_timestamp`
   - Dodaj kolumnę `data_quality_flag` = "VALID"
3. Zapisz jako `customers_silver` w Silver schema

**Oczekiwany rezultat:**
- Oczyszczone, zwalidowane dane w Silver
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 4.2 - Silver Layer

spark.sql(f"USE SCHEMA {SILVER_SCHEMA}")

# Wczytaj dane z Bronze
customers_bronze_df = spark.table(customers_bronze_table)

# Silver transformations
customers_silver = (
    customers_bronze_df
    # Deduplikacja
    .____([____])
    
    # Walidacja NOT NULL
    .filter(F.col("____").isNotNull())
    .filter(F.col("____").isNotNull())
    
    # Standaryzacja email
    .withColumn("email", F.____(F.col("email")))
    
    # Walidacja wieku
    .filter((F.col("____") > ____) & (F.col("____") < ____))
    
    # Silver metadata
    .withColumn("____", F.current_timestamp())
    .withColumn("____", F.lit("VALID"))
)

print("=== Silver layer - cleansed data ===")
display(customers_silver.limit(5))

# Zapisz do Silver
customers_silver_table = f"{SILVER_SCHEMA}.customers_silver"

(
    customers_silver
    .write
    .format("____")
    .mode("____")
    .option("overwriteSchema", "true")
    .saveAsTable(____)
)

bronze_count = customers_bronze_df.count()
silver_count = spark.table(customers_silver_table).count()

print(f"\n✓ Silver layer utworzony: {customers_silver_table}")
print(f"Bronze records: {bronze_count}")
print(f"Silver records: {silver_count}")
print(f"Filtered out: {bronze_count - silver_count} records")
</VSCode.Cell>
<VSCode.Cell language="markdown">
### Zadanie 4.3: Gold Layer - Business Aggregates

**Instrukcje:**
1. Wczytaj dane z Silver: `customers_silver`
2. Utwórz agregację klientów per kraj:
   - `customer_count`: liczba klientów
   - `avg_age`: średni wiek
   - `min_registration_date`: najwcześniejsza rejestracja
   - `max_registration_date`: najpóźniejsza rejestracja
3. Dodaj `gold_created_timestamp`
4. Zapisz jako `customer_summary_by_country` w Gold schema

**Oczekiwany rezultat:**
- Tabela Gold z agregacjami biznesowymi
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 4.3 - Gold Layer

spark.sql(f"USE SCHEMA {GOLD_SCHEMA}")

# Wczytaj dane z Silver
customers_silver_df = spark.table(customers_silver_table)

# Gold aggregation: Summary per country
customer_summary = (
    customers_silver_df
    .groupBy("____")
    .agg(
        F.count("____").alias("____"),
        F.avg("____").alias("____"),
        F.min("____").alias("____"),
        F.max("____").alias("____")
    )
    .withColumn("____", F.current_timestamp())
    .orderBy("customer_count", ascending=False)
)

print("=== Gold layer - Customer Summary by Country ===")
display(customer_summary)

# Zapisz do Gold
gold_table = f"{GOLD_SCHEMA}.customer_summary_by_country"

(
    customer_summary
    .write
    .format("____")
    .mode("____")
    .option("overwriteSchema", "true")
    .saveAsTable(____)
)

print(f"\n✓ Gold layer utworzony: {gold_table}")
print(f"Liczba krajów: {spark.table(gold_table).count()}")
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Zadanie 5: Optymalizacja tabel Delta (15 minut)

**Cel:** Zastosowanie technik optymalizacji: OPTIMIZE, ZORDER, VACUUM.

### Zadanie 5.1: OPTIMIZE - Compaction małych plików

**Instrukcje:**
1. Sprawdź liczbę plików w tabeli `customers_silver` używając `DESCRIBE DETAIL`
2. Uruchom `OPTIMIZE` na tabeli
3. Sprawdź ponownie liczbę plików - powinna się zmniejszyć

**Wskazówki:**
- `DESCRIBE DETAIL table_name`
- `OPTIMIZE table_name`
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 5.1 - OPTIMIZE

# Stan przed OPTIMIZE
print("=== Stan przed OPTIMIZE ===")
detail_before = spark.sql(f"____ ____ {customers_silver_table}")
display(detail_before.select("numFiles", "sizeInBytes"))

num_files_before = detail_before.collect()[0]["numFiles"]
print(f"Liczba plików przed: {num_files_before}")

# Uruchom OPTIMIZE
print("\n=== Uruchamianie OPTIMIZE ===")
optimize_result = spark.sql(f"____ {____}")
display(optimize_result)

# Stan po OPTIMIZE
print("\n=== Stan po OPTIMIZE ===")
detail_after = spark.sql(f"DESCRIBE DETAIL {customers_silver_table}")
display(detail_after.select("numFiles", "sizeInBytes"))

num_files_after = detail_after.collect()[0]["numFiles"]
print(f"Liczba plików po: {num_files_after}")
print(f"Redukcja: {num_files_before - num_files_after} plików")

print("✓ OPTIMIZE zakończony!")
</VSCode.Cell>
<VSCode.Cell language="markdown">
### Zadanie 5.2: ZORDER - Clustering dla lepszego data skipping

**Instrukcje:**
1. Uruchom `OPTIMIZE` z `ZORDER BY` na tabeli `orders_bronze`
2. Użyj `ZORDER BY (customer_id)` - kolumna często używana w filtrach
3. Sprawdź metryki optymalizacji

**Wskazówki:**
- `OPTIMIZE table_name ZORDER BY (column1, column2)`
</VSCode.Cell>
<VSCode.Cell language="python">
# TODO: Zadanie 5.2 - ZORDER

# Uruchom OPTIMIZE z ZORDER na customer_id
print("=== OPTIMIZE + ZORDER BY customer_id ===")

zorder_result = spark.sql(f"""
    ____ {orders_table}
    ____ BY (____)
""")

display(zorder_result)

print("✓ ZORDER zakończony - dane są teraz lepiej zorganizowane dla queries po customer_id!")
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Walidacja i weryfikacja

### Checklist - Co powinieneś uzyskać:
- [ ] Tabela `products_bronze` utworzona z operacjami UPDATE i DELETE
- [ ] Tabela `orders_bronze` z zastosowanym MERGE INTO
- [ ] Time Travel działa - można odczytać starą wersję `products_bronze`
- [ ] Pipeline Medallion: `customers_bronze` → `customers_silver` → `customer_summary_by_country`
- [ ] Tabele zoptymalizowane (OPTIMIZE, ZORDER)

### Komendy weryfikacyjne:
</VSCode.Cell>
<VSCode.Cell language="python">
# Weryfikacja wyników

print("=== WERYFIKACJA WYNIKÓW ===\n")

# 1. Sprawdź tabele Bronze
print("1. Tabele Bronze:")
bronze_tables = spark.sql(f"SHOW TABLES IN {BRONZE_SCHEMA}").filter(F.col("tableName").like("%bronze%"))
display(bronze_tables)

# 2. Sprawdź tabele Silver
print("\n2. Tabele Silver:")
silver_tables = spark.sql(f"SHOW TABLES IN {SILVER_SCHEMA}").filter(F.col("tableName").like("%silver%"))
display(silver_tables)

# 3. Sprawdź tabele Gold
print("\n3. Tabele Gold:")
gold_tables = spark.sql(f"SHOW TABLES IN {GOLD_SCHEMA}")
display(gold_tables)

# 4. Sprawdź historię dla products_bronze
print("\n4. Historia products_bronze (powinny być operacje: WRITE, UPDATE, DELETE):")
history = spark.sql(f"DESCRIBE HISTORY {products_table}")
display(history.select("version", "timestamp", "operation", "operationMetrics").limit(10))

# 5. Sprawdź dane Gold
print("\n5. Przykładowe dane Gold:")
display(spark.table(gold_table).limit(10))

print("\n✓ Wszystkie testy przeszły pomyślnie!")
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Podsumowanie

**W tym warsztacie nauczyłeś się:**

✅ **Delta Lake CRUD Operations:**
- CREATE TABLE z różnych formatów źródłowych
- UPDATE - aktualizacja rekordów
- DELETE - usuwanie rekordów
- MERGE INTO - upsert (update + insert)

✅ **Time Travel:**
- DESCRIBE HISTORY - historia wersji
- VERSION AS OF - odczytywanie starych wersji
- Możliwość audytu i rollback

✅ **Medallion Architecture:**
- Bronze: Raw data + audit metadata (immutable, append-only)
- Silver: Cleansed, validated, deduplicated
- Gold: Business aggregates, KPI models

✅ **Optymalizacja:**
- OPTIMIZE - compaction małych plików
- ZORDER - clustering dla lepszego data skipping
- DESCRIBE DETAIL - monitoring stanu tabel

**Kluczowe wnioski:**
1. Delta Lake zapewnia ACID transactions dla data lake
2. Time Travel umożliwia audyt i recovery bez backupów
3. Medallion Architecture separuje raw data od business logic
4. Regularna optymalizacja (OPTIMIZE) jest kluczowa dla performance

**Następne kroki:**
- **Kolejny warsztat**: 02_ingestion_pipeline_workshop.ipynb - COPY INTO, Auto Loader, Streaming
- **Materiały dodatkowe**: Delta Lake Documentation, Medallion Architecture Guide
</VSCode.Cell>
<VSCode.Cell language="markdown">
---

## Cleanup

Opcjonalnie: usuń utworzone tabele po zakończeniu warsztatu:
</VSCode.Cell>
<VSCode.Cell language="python">
# Opcjonalne czyszczenie zasobów testowych
# UWAGA: Uruchom tylko jeśli chcesz usunąć wszystkie utworzone dane

# spark.sql(f"DROP TABLE IF EXISTS {products_table}")
# spark.sql(f"DROP TABLE IF EXISTS {orders_table}")
# spark.sql(f"DROP TABLE IF EXISTS {customers_bronze_table}")
# spark.sql(f"DROP TABLE IF EXISTS {customers_silver_table}")
# spark.sql(f"DROP TABLE IF EXISTS {gold_table}")

# spark.catalog.clearCache()
# print("Zasoby zostały wyczyszczone")
</VSCode.Cell>
```